In [1]:
#main
import gradio as gr
from groq import Groq

# Initialize the Groq client
client = Groq(api_key="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")

# Storage to retain user data across screens
user_data = {}

# First screen: Collect user details
def collect_user_details(name, age, std, region, interests, hobbies, ambition, skills, grade, aim, friends, study_hours, parent_hours):
    if not name or not age:
        return "Error: Name and Age are mandatory. Please fill them in.", gr.update(visible=True), gr.update(visible=False)

    global user_data
    user_data = {
        "Name": name,
        "Age": age,
        "Standard": std,
        "Region": region,
        "Interests": interests,
        "Hobbies": hobbies,
        "Future Ambition": ambition,
        "Skills to Improve": skills,
        "Grade": grade,
        "Academic Aim": aim,
        "Number of Friends": friends,
        "Study Hours": study_hours,
        "Parent Hours": parent_hours,
    }
    return "User details collected. Proceeding to the next step.", gr.update(visible=False), gr.update(visible=True)

# Second screen: Collect study topic and context
def collect_topic_details(topic, intention, study_time, personalize):
    user_data["Topic"] = topic
    user_data["Intention"] = intention
    user_data["Study Time"] = study_time
    user_data["Personalize"] = personalize
    return f"Preparing content for {topic} based on your input.", gr.update(visible=False), gr.update(visible=True)

# Third screen: Generate and present content
def generate_content():
    topic = user_data.get("Topic", "No topic provided.")

    # Generate the main content
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": topic}],
        model="llama3-8b-8192",
    )
    content = response.choices[0].message.content

    # Generate additional outputs
    fact_prompt = f"Provide an interesting thought or fact about: {topic}"
    fact_response = client.chat.completions.create(
        messages=[{"role": "user", "content": fact_prompt}],
        model="llama3-8b-8192",
    )
    interesting_fact = fact_response.choices[0].message.content[:200]

    history_prompt = f"Provide a historical connotation about: {topic} in 3 sentences or less."
    history_response = client.chat.completions.create(
        messages=[{"role": "user", "content": history_prompt}],
        model="llama3-8b-8192",
    )
    historical_connotation = history_response.choices[0].message.content[:300]

    importance_prompt = f"Explain the importance of {topic} for competitive or board exams in 2 sentences."
    importance_response = client.chat.completions.create(
        messages=[{"role": "user", "content": importance_prompt}],
        model="llama3-8b-8192",
    )
    importance_exams = importance_response.choices[0].message.content[:200]

    words_prompt = f"List key important words from the explanation of: {topic}."
    words_response = client.chat.completions.create(
        messages=[{"role": "user", "content": words_prompt}],
        model="llama3-8b-8192",
    )
    important_words = words_response.choices[0].message.content

    youtube_prompt = f"Suggest up to 2 YouTube links to learn more about: {topic}."
    youtube_response = client.chat.completions.create(
        messages=[{"role": "user", "content": youtube_prompt}],
        model="llama3-8b-8192",
    )
    youtube_links = youtube_response.choices[0].message.content

    articles_prompt = f"Suggest up to 2 scientific articles or papers about: {topic}."
    articles_response = client.chat.completions.create(
        messages=[{"role": "user", "content": articles_prompt}],
        model="llama3-8b-8192",
    )
    scientific_articles = articles_response.choices[0].message.content

    return (
        content,
        interesting_fact,
        historical_connotation,
        importance_exams,
        important_words,
        youtube_links,
        scientific_articles,
    )

# Gradio interface setup
with gr.Blocks() as demo:
    # First screen
    with gr.Group(visible=True) as user_details_section:
        gr.Markdown("### Step 1: Please enter your details")
        name = gr.Textbox(label="Name", placeholder="Enter your name")
        age = gr.Textbox(label="Age", placeholder="Enter your age")
        std = gr.Textbox(label="Standard", placeholder="Enter your class/grade")
        region = gr.Textbox(label="Region", placeholder="Enter your region")
        interests = gr.Textbox(label="Interests", placeholder="E.g., Math, Science, Art")
        hobbies = gr.Textbox(label="Hobbies", placeholder="E.g., Reading, Painting")
        ambition = gr.Textbox(label="Future Ambition", placeholder="E.g., Doctor, Engineer")
        skills = gr.Textbox(label="Skills to Improve", placeholder="E.g., Problem-solving")
        grade = gr.Textbox(label="Previous Exam Grade", placeholder="Enter your previous grade/marks")
        aim = gr.Textbox(label="Academic Aim", placeholder="E.g., Top 10 rank, pass exam")
        friends = gr.Textbox(label="Number of Friends", placeholder="Enter number of friends")
        study_hours = gr.Textbox(label="Study Hours", placeholder="Hours you study daily")
        parent_hours = gr.Textbox(label="Parent Support Hours", placeholder="Hours your parents support you")
        submit_user_details = gr.Button("Submit")
        user_feedback = gr.Textbox(label="Status", interactive=False)

    # Second screen
    with gr.Group(visible=False) as topic_section:
        gr.Markdown("### Step 2: Enter details about the topic you want to study")
        topic = gr.Textbox(label="Topic", placeholder="Enter topic to learn")
        intention = gr.Radio(
            ["Board Exam", "Competitive Exam", "Fun", "Curiosity", "Just Learning"],
            label="Study Intention"
        )
        study_time = gr.Textbox(label="Study Count", placeholder="How many times studied this topic?")
        personalize = gr.Checkbox(label="Personalize the content")
        submit_topic = gr.Button("Submit")
        topic_feedback = gr.Textbox(label="Status", interactive=False)

    # Third screen: Updated grid layout
    with gr.Group(visible=False) as content_section:
        gr.Markdown("### Step 3: Here is your personalized study content")
        with gr.Row():
            with gr.Column():
                content_output = gr.Textbox(label="Generated Content", interactive=False, lines=10)
                interesting_fact_output = gr.Textbox(label="Interesting Fact", interactive=False, lines=2)
                historical_connotation_output = gr.Textbox(label="Historical Connotation", interactive=False, lines=3)
            with gr.Column():
                importance_exams_output = gr.Textbox(label="Importance for Exams", interactive=False, lines=2)
                important_words_output = gr.Textbox(label="Important Words", interactive=False, lines=2)
                youtube_links_output = gr.Textbox(label="YouTube Links", interactive=False, lines=2)
                scientific_articles_output = gr.Textbox(label="Scientific Articles", interactive=False, lines=2)
        generate_button = gr.Button("Generate Content")
        previous_button = gr.Button("Previous")  # Add Previous Button

    # Navigation and logic
    submit_user_details.click(
        collect_user_details,
        inputs=[name, age, std, region, interests, hobbies, ambition, skills, grade, aim, friends, study_hours, parent_hours],
        outputs=[user_feedback, user_details_section, topic_section]
    )
    submit_topic.click(
        collect_topic_details,
        inputs=[topic, intention, study_time, personalize],
        outputs=[topic_feedback, topic_section, content_section]
    )
    generate_button.click(
        generate_content,
        outputs=[
            content_output,
            interesting_fact_output,
            historical_connotation_output,
            importance_exams_output,
            important_words_output,
            youtube_links_output,
            scientific_articles_output
        ]
    )

    # Update the Previous button's functionality
    previous_button.click(
        lambda: (gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)),  # Hide current section, show previous section
        inputs=None,
        outputs=[content_section, topic_section, user_details_section]  # Toggle visibility of content_section and topic_section
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.


In [2]:
# Gradio interface setup
with gr.Blocks() as demo:
    # First screen
    with gr.Group(visible=True) as user_details_section:
        gr.Markdown("### Step 1: Please enter your details")
        name = gr.Textbox(label="Name", placeholder="Enter your name")
        age = gr.Textbox(label="Age", placeholder="Enter your age")
        std = gr.Textbox(label="Standard", placeholder="Enter your class/grade")
        region = gr.Textbox(label="Region", placeholder="Enter your region")
        interests = gr.Textbox(label="Interests", placeholder="E.g., Math, Science, Art")
        hobbies = gr.Textbox(label="Hobbies", placeholder="E.g., Reading, Painting")
        ambition = gr.Textbox(label="Future Ambition", placeholder="E.g., Doctor, Engineer")
        skills = gr.Textbox(label="Skills to Improve", placeholder="E.g., Problem-solving")
        grade = gr.Textbox(label="Previous Exam Grade", placeholder="Enter your previous grade/marks")
        aim = gr.Textbox(label="Academic Aim", placeholder="E.g., Top 10 rank, pass exam")
        friends = gr.Textbox(label="Number of Friends", placeholder="Enter number of friends")
        study_hours = gr.Textbox(label="Study Hours", placeholder="Hours you study daily")
        parent_hours = gr.Textbox(label="Parent Support Hours", placeholder="Hours your parents support you")
        submit_user_details = gr.Button("Submit")
        user_feedback = gr.Textbox(label="Status", interactive=False)

    # Second screen
    with gr.Group(visible=False) as topic_section:
        gr.Markdown("### Step 2: Enter details about the topic you want to study")
        topic = gr.Textbox(label="Topic", placeholder="Enter topic to learn")
        intention = gr.Radio(
            ["Board Exam", "Competitive Exam", "Fun", "Curiosity", "Just Learning"],
            label="Study Intention"
        )
        study_time = gr.Textbox(label="Study Count", placeholder="How many times studied this topic?")
        personalize = gr.Checkbox(label="Personalize the content")
        submit_topic = gr.Button("Submit")
        topic_feedback = gr.Textbox(label="Status", interactive=False)

    # Third screen: Updated grid layout
    with gr.Group(visible=False) as content_section:
        gr.Markdown("### Step 3: Here is your personalized study content")
        with gr.Row():
            with gr.Column():
                interesting_fact_output = gr.Textbox(label="Interesting Fact", interactive=False, lines=2)
                historical_connotation_output = gr.Textbox(label="Historical Connotation", interactive=False, lines=3)
            with gr.Column():
                importance_exams_output = gr.Textbox(label="Importance for Exams", interactive=False, lines=2)
                important_words_output = gr.Textbox(label="Important Words", interactive=False, lines=2)
                youtube_links_output = gr.Textbox(label="YouTube Links", interactive=False, lines=2)
                scientific_articles_output = gr.Textbox(label="Scientific Articles", interactive=False, lines=2)
        generate_button = gr.Button("Generate Content")
        previous_button = gr.Button("Previous")  # Add Previous Button

    # Navigation and logic
    submit_user_details.click(
        collect_user_details,
        inputs=[name, age, std, region, interests, hobbies, ambition, skills, grade, aim, friends, study_hours, parent_hours],
        outputs=[user_feedback, user_details_section, topic_section]
    )
    submit_topic.click(
        collect_topic_details,
        inputs=[topic, intention, study_time, personalize],
        outputs=[topic_feedback, topic_section, content_section]
    )
    generate_button.click(
        generate_content,
        outputs=[
            interesting_fact_output,
            historical_connotation_output,
            importance_exams_output,
            important_words_output,
            youtube_links_output,
            scientific_articles_output
        ]
    )

    # Update the Previous button's functionality
    previous_button.click(
        lambda: (gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)),  # Hide current section, show previous section
        inputs=None,
        outputs=[content_section, topic_section, user_details_section]  # Toggle visibility of content_section and topic_section
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.


C:\Users\mark kevin\anaconda3\Lib\site-packages\gradio\blocks.py:1748: UserWarning: A function (generate_content) returned too many output values (needed: 6, returned: 7). Ignoring extra values.
    Output components:
        [textbox, textbox, textbox, textbox, textbox, textbox]
    Output values returned:
        ["The current:

* What is the current time?", "Here's something interesting about currents:

Did you know that the Gulf Stream, a powerful ocean current in the North Atlantic, is responsible for keeping Western Europe relatively mild and temperate", "The concept of "current" in terms of electricity has its roots in the 1830s, when British scientist Michael Faraday discovered the principle of electromagnetic induction. This breakthrough led to the development of the first practical electrical current, which was harnessed and distributed by invent", "The concept of current is crucial in competitive and board exams as it is a fundamental principle in physics, and understanding 